In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace
from ngsolve.meshes import MakeStructured2DMesh


importing ngsxfem-2.1.2504


In [2]:
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
h0 = 1/10

In [3]:
# print(help(MakeStructured2DMesh))

In [13]:
# Construct background mesh
# Geometry and Mesh
square = SplineGeometry()
# square.AddRectangle((-1.5, -1.5), (1.5, 1.5), bc=1)
# square.AddRectangle((-1, -1), (1, 1), bc=1)
square.AddRectangle((-2, -2), (2, 2), bc=1)
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# mesh = MakeStructured2DMesh(quads=False, nx=8, ny=8)
# Draw(mesh)

In [14]:
# levelset = sqrt(x**2 + y**2) - 0.5 # 圆
#### 椭圆 ####
# a = 1/4
# b = 1/3
# levelset = (x-1/2)**2/a**2 + (y-1/2)**2/b**2 -1

#### 心型线 ####
levelset = (x**2+y**2-1)**3 - x**2*y**3


mask = IfPos(levelset,1,0)
Draw(mask,mesh)



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [15]:
# Draw(levelset,mesh,min = -1, max = 0)

In [16]:
# Higher order level set approximation
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2, threshold=0.1,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

# lsetp1 = GridFunction(H1(mesh,order=1))
# InterpolateToP1(levelset,lsetp1)
# DrawDC(lsetp1,-1,1,mesh)
# mask2 = IfPos(lsetp1,1,0)
# Draw(mask2,mesh)

In [17]:
# levelset_domain = IfPos(levelset,0,1)
# Draw(levelset_domain,mesh)

In [18]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
neg = ci.GetElementsOfType(NEG)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
# Draw(BitArrayCF(neg),mesh)
# Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


In [10]:
print(type(ba_facets))

<class 'pyngcore.pyngcore.BitArray'>


In [19]:
vtk = VTKOutput(mesh,[levelset,BitArrayCF(hasneg),BitArrayCF(hasif)],['levelset','active_mesh','boundary'],"/mnt/d/ngs_output/Biot_Brinkman/heart_domain",subdivision=0)
# vtk = VTKOutput(mesh,[levelset_domain],['levelset_domain'],"/mnt/d/ngs_output/Biot_Brinkman/levelset_domain",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(neg)],['uncutElements'],"/mnt/d/ngs_output/Biot_Brinkman/uncutElements",subdivision=0)
# vtk = VTKOutput(mesh,[BitArrayCF(ba_facets)],['ghost_faces'],"/mnt/d/ngs_output/Biot_Brinkman/ghost_faces",subdivision=0)
vtk.Do()   #输出网格

'/mnt/d/ngs_output/Biot_Brinkman/heart_domain'

In [ ]:
print(help(Draw))